In [1]:
import numpy as np
import pandas as pd

In [2]:
# load all datasets
ben_df = pd.read_csv('./US Healthcare Dataset/Train_Beneficiarydata-1542865627584.csv')
inp_df = pd.read_csv('./US Healthcare Dataset/Train_Inpatientdata-1542865627584.csv')
out_df = pd.read_csv('./US Healthcare Dataset/Train_Outpatientdata-1542865627584.csv')
label_df = pd.read_csv('./US Healthcare Dataset/Train-1542865627584.csv')

print(f"Beneficiary Data Shape: {ben_df.shape}")
print(f"Inpatient Data Shape: {inp_df.shape}")
print(f"Outpatient Data Shape: {out_df.shape}")
print(f"Labels Data Shape: {label_df.shape}")

Beneficiary Data Shape: (138556, 25)
Inpatient Data Shape: (40474, 30)
Outpatient Data Shape: (517737, 27)
Labels Data Shape: (5410, 2)


In [3]:
ben_df.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


In [4]:
inp_df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [5]:
out_df.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [6]:
label_df.head()

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [7]:
ben_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   DOB                              138556 non-null  object
 2   DOD                              1421 non-null    object
 3   Gender                           138556 non-null  int64 
 4   Race                             138556 non-null  int64 
 5   RenalDiseaseIndicator            138556 non-null  object
 6   State                            138556 non-null  int64 
 7   County                           138556 non-null  int64 
 8   NoOfMonths_PartACov              138556 non-null  int64 
 9   NoOfMonths_PartBCov              138556 non-null  int64 
 10  ChronicCond_Alzheimer            138556 non-null  int64 
 11  ChronicCond_Heartfailure         138556 non-null  int64 
 12  ChronicCond_Kidn

In [8]:
inp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BeneID                  40474 non-null  object 
 1   ClaimID                 40474 non-null  object 
 2   ClaimStartDt            40474 non-null  object 
 3   ClaimEndDt              40474 non-null  object 
 4   Provider                40474 non-null  object 
 5   InscClaimAmtReimbursed  40474 non-null  int64  
 6   AttendingPhysician      40362 non-null  object 
 7   OperatingPhysician      23830 non-null  object 
 8   OtherPhysician          4690 non-null   object 
 9   AdmissionDt             40474 non-null  object 
 10  ClmAdmitDiagnosisCode   40474 non-null  object 
 11  DeductibleAmtPaid       39575 non-null  float64
 12  DischargeDt             40474 non-null  object 
 13  DiagnosisGroupCode      40474 non-null  object 
 14  ClmDiagnosisCode_1      40474 non-null

In [9]:
out_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  517737 non-null  object 
 1   ClaimID                 517737 non-null  object 
 2   ClaimStartDt            517737 non-null  object 
 3   ClaimEndDt              517737 non-null  object 
 4   Provider                517737 non-null  object 
 5   InscClaimAmtReimbursed  517737 non-null  int64  
 6   AttendingPhysician      516341 non-null  object 
 7   OperatingPhysician      90617 non-null   object 
 8   OtherPhysician          195046 non-null  object 
 9   ClmDiagnosisCode_1      507284 non-null  object 
 10  ClmDiagnosisCode_2      322357 non-null  object 
 11  ClmDiagnosisCode_3      203257 non-null  object 
 12  ClmDiagnosisCode_4      125596 non-null  object 
 13  ClmDiagnosisCode_5      74344 non-null   object 
 14  ClmDiagnosisCode_6  

In [10]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provider        5410 non-null   object
 1   PotentialFraud  5410 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB


In [11]:
# Checking the target class balance of the target variable
print("TARGET VARIABLE DISTRIBUTION")
print(label_df['PotentialFraud'].value_counts())
# normalize returns the proportion of each category
print(label_df['PotentialFraud'].value_counts(normalize=True) * 100)

TARGET VARIABLE DISTRIBUTION
PotentialFraud
No     4904
Yes     506
Name: count, dtype: int64
PotentialFraud
No     90.64695
Yes     9.35305
Name: proportion, dtype: float64


In [12]:
# Confirm BeneID links Beneficiary and Inpatient/Outpatient
# Check if all BeneIDs in Inpatient exist in Beneficiary
inp_beneids = set(inp_df['BeneID'].unique())
ben_beneids = set(ben_df['BeneID'].unique())
inp_not_in_ben = inp_beneids - ben_beneids
print(f"Inpatient BeneIDs not in Beneficiary: {len(inp_not_in_ben)}")
if len(inp_not_in_ben) > 0:
    print(f"Sample: {list(inp_not_in_ben)[:5]}")

# Check if all BeneIDs in Outpatient exist in Beneficiary
out_beneids = set(out_df['BeneID'].unique())
out_not_in_ben = out_beneids - ben_beneids
print(f"Outpatient BeneIDs not in Beneficiary: {len(out_not_in_ben)}")
if len(out_not_in_ben) > 0:
    print(f"Sample: {list(out_not_in_ben)[:5]}")

# Check overlap
print(f"\nUnique BeneIDs in Beneficiary: {len(ben_beneids)}")
print(f"Unique BeneIDs in Inpatient: {len(inp_beneids)}")
print(f"Unique BeneIDs in Outpatient: {len(out_beneids)}")
print(f"BeneIDs in both Inpatient and Outpatient: {len(inp_beneids & out_beneids)}")

Inpatient BeneIDs not in Beneficiary: 0
Outpatient BeneIDs not in Beneficiary: 0

Unique BeneIDs in Beneficiary: 138556
Unique BeneIDs in Inpatient: 31289
Unique BeneIDs in Outpatient: 133980
BeneIDs in both Inpatient and Outpatient: 26713


In [13]:
# Confirm ClaimID is unique per claim
# creates a combined claims table with a column determining whether each claim is from inpatient or outpatient
all_claims = pd.concat([
    inp_df[['ClaimID', 'BeneID', 'Provider']].assign(ClaimType='Inpatient'),
    out_df[['ClaimID', 'BeneID', 'Provider']].assign(ClaimType='Outpatient')
], ignore_index=True)

# Check for any duplicate ClaimIDs
duplicate_claimids = all_claims[all_claims.duplicated(subset=['ClaimID'], keep=False)]
print(f"Total unique ClaimIDs: {all_claims['ClaimID'].nunique()}")
print(f"Total rows with claims: {len(all_claims)}")
print(f"Duplicate ClaimIDs found: {len(duplicate_claimids)}")

Total unique ClaimIDs: 558211
Total rows with claims: 558211
Duplicate ClaimIDs found: 0


In [14]:
# Confirm Provider links claims and labels

# Get unique providers from claims
inp_providers = set(inp_df['Provider'].unique())
out_providers = set(out_df['Provider'].unique())
all_claim_providers = inp_providers | out_providers

# Get providers from labels
label_providers = set(label_df['Provider'].unique())

print(f"Unique Providers in Inpatient: {len(inp_providers)}")
print(f"Unique Providers in Outpatient: {len(out_providers)}")
print(f"Unique Providers in all claims: {len(all_claim_providers)}")
print(f"Unique Providers in labels: {len(label_providers)}")

# Check if all label providers appear in claims
providers_in_labels_not_in_claims = label_providers - all_claim_providers
print(f"\nProviders in labels but NOT in claims: {len(providers_in_labels_not_in_claims)}")

# Check if there are providers in claims not in labels
providers_in_claims_not_in_labels = all_claim_providers - label_providers
print(f"Providers in claims but NOT in labels: {len(providers_in_claims_not_in_labels)}")

Unique Providers in Inpatient: 2092
Unique Providers in Outpatient: 5012
Unique Providers in all claims: 5410
Unique Providers in labels: 5410

Providers in labels but NOT in claims: 0
Providers in claims but NOT in labels: 0


In [15]:
# Is there any claims that has Provider missing?
inp_missing_provider = inp_df['Provider'].isna().sum()
out_missing_provider = out_df['Provider'].isna().sum()

print(f"Inpatient claims with missing Provider: {inp_missing_provider}")
print(f"Outpatient claims with missing Provider: {out_missing_provider}")

Inpatient claims with missing Provider: 0
Outpatient claims with missing Provider: 0


In [16]:
# Does every Provider in labels appear in claims?
providers_missing_from_claims = label_providers - all_claim_providers
print(f"Providers in train labels: {len(label_providers)}")
print(f"Providers in claims: {len(all_claim_providers)}")
print(f"Providers in labels but NOT in claims: {len(providers_missing_from_claims)}")

Providers in train labels: 5410
Providers in claims: 5410
Providers in labels but NOT in claims: 0


In [17]:
# Is there any Beneficiaries that never appear in claims?
# Get all BeneIDs from claims
all_claim_beneids = set(all_claims['BeneID'].unique())

# Find beneficiaries that never appear in claims
beneficiaries_without_claims = ben_beneids - all_claim_beneids

print(f"Total Beneficiaries: {len(ben_beneids)}")
print(f"Beneficiaries with claims: {len(all_claim_beneids)}")
print(f"Beneficiaries WITHOUT any claims: {len(beneficiaries_without_claims)}")

Total Beneficiaries: 138556
Beneficiaries with claims: 138556
Beneficiaries WITHOUT any claims: 0


In [18]:
# missing values analysis

def analyze_missing_pattern(df, df_name):
    print(f"\n{df_name} Dataset Missing Values:")
    print("-" * 60)
    
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percentage': missing_pct
    })
    missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df.to_string())
    else:
        print("No missing values found")
    
    return missing_df

# Analyze each dataset
ben_missing = analyze_missing_pattern(ben_df, "Beneficiary")
inp_missing = analyze_missing_pattern(inp_df, "Inpatient")
out_missing = analyze_missing_pattern(out_df, "Outpatient")
label_missing = analyze_missing_pattern(label_df, "Labels")


Beneficiary Dataset Missing Values:
------------------------------------------------------------
     Missing_Count  Missing_Percentage
DOD         137135               98.97

Inpatient Dataset Missing Values:
------------------------------------------------------------
                     Missing_Count  Missing_Percentage
ClmProcedureCode_6           40474              100.00
ClmProcedureCode_5           40465               99.98
ClmProcedureCode_4           40358               99.71
ClmProcedureCode_3           39509               97.62
ClmDiagnosisCode_10          36547               90.30
OtherPhysician               35784               88.41
ClmProcedureCode_2           35020               86.52
ClmProcedureCode_1           17326               42.81
OperatingPhysician           16644               41.12
ClmDiagnosisCode_9           13497               33.35
ClmDiagnosisCode_8            9942               24.56
ClmDiagnosisCode_7            7258               17.93
ClmDiagnosisC

In [19]:
def analyze_diag_missing(df, dataset):
    # Identify diagnosis code columns
    print(f"\n{dataset} Diagnosis Codes ({len(diag_cols)} columns):")
    print("-" * 60)

    # Missing value summary
    missing_stats = []
    for col in sorted(diag_cols, key=lambda x: int(x.split('_')[1])):
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df) * 100)

        missing_stats.append({
            "Column": col,
            "Missing_Count": missing_count,
            "Missing_Percentage": f"{missing_pct:.2f}%",
            "Non_Null_Count": df[col].notna().sum()
        })

    diag_missing_df = pd.DataFrame(missing_stats)
    print(diag_missing_df.to_string(index=False))

    # Diagnosis count pattern (per claim)
    diag_count = df[diag_cols].notna().sum(axis=1)
    print(f"\n{dataset} - Number of diagnosis codes per claim:")
    print(diag_count.value_counts().sort_index().head(15))

    return diag_missing_df, diag_count

inp_diag_df, inp_diag_count = analyze_diag_missing(inp_df, "Inpatient")
out_diag_df, out_diag_count = analyze_diag_missing(out_df, "Outpatient")


Inpatient Diagnosis Codes (10 columns):
------------------------------------------------------------
             Column  Missing_Count Missing_Percentage  Non_Null_Count
 ClmDiagnosisCode_1              0              0.00%           40474
 ClmDiagnosisCode_2            226              0.56%           40248
 ClmDiagnosisCode_3            676              1.67%           39798
 ClmDiagnosisCode_4           1534              3.79%           38940
 ClmDiagnosisCode_5           2894              7.15%           37580
 ClmDiagnosisCode_6           4838             11.95%           35636
 ClmDiagnosisCode_7           7258             17.93%           33216
 ClmDiagnosisCode_8           9942             24.56%           30532
 ClmDiagnosisCode_9          13497             33.35%           26977
ClmDiagnosisCode_10          36547             90.30%            3927

Inpatient - Number of diagnosis codes per claim:
1       226
2       450
3       858
4      1360
5      1944
6      2420
7     

In [20]:
def analyze_proc_missing(df, dataset):
    # Identify procedure code columns
    proc_cols = [col for col in df.columns if "ClmProcedureCode" in col]
    print(f"\n{dataset} Procedure Codes ({len(proc_cols)} columns):")
    print("-" * 60)

    # Missing statistics for each procedure column
    proc_missing_stats = []
    for col in sorted(proc_cols):
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df) * 100)

        proc_missing_stats.append({
            "Column": col,
            "Missing_Count": missing_count,
            "Missing_Percentage": f"{missing_pct:.2f}%",
            "Non_Null_Count": df[col].notna().sum()
        })

    proc_missing_df = pd.DataFrame(proc_missing_stats)
    print(proc_missing_df.to_string(index=False))

    # Procedure code count per claim
    proc_count = df[proc_cols].notna().sum(axis=1)

    print(f"\n{dataset} - Number of procedure codes per claim:")
    print(proc_count.value_counts().sort_index())

    return proc_missing_df, proc_count

inp_proc_df, inp_proc_count = analyze_proc_missing(inp_df, "Inpatient")
out_proc_df, out_proc_count = analyze_proc_missing(out_df, "Outpatient")


Inpatient Procedure Codes (6 columns):
------------------------------------------------------------
            Column  Missing_Count Missing_Percentage  Non_Null_Count
ClmProcedureCode_1          17326             42.81%           23148
ClmProcedureCode_2          35020             86.52%            5454
ClmProcedureCode_3          39509             97.62%             965
ClmProcedureCode_4          40358             99.71%             116
ClmProcedureCode_5          40465             99.98%               9
ClmProcedureCode_6          40474            100.00%               0

Inpatient - Number of procedure codes per claim:
0    17326
1    17694
2     4489
3      849
4      107
5        9
Name: count, dtype: int64

Outpatient Procedure Codes (6 columns):
------------------------------------------------------------
            Column  Missing_Count Missing_Percentage  Non_Null_Count
ClmProcedureCode_1         517575             99.97%             162
ClmProcedureCode_2         517701 

In [21]:
def analyze_reimbursement(df, cols, label="Dataset"):
    print(f"\n{label} Reimbursement:")
    print("-" * 60)

    for col in cols:
        missing = df[col].isna().sum()
        missing_pct = (missing / len(df) * 100)

        print(f"{col}:")
        print(f"  Missing: {missing} ({missing_pct:.2f}%)")

        # Only print stats if no missing values
        if missing == 0:
            print(f"  Min: {df[col].min():,.2f}")
            print(f"  Max: {df[col].max():,.2f}")
            print(f"  Mean: {df[col].mean():,.2f}")
            print(f"  Median: {df[col].median():,.2f}")

inp_reimb_cols = ['InscClaimAmtReimbursed', 'DeductibleAmtPaid']
analyze_reimbursement(inp_df, inp_reimb_cols, label="Inpatient")

out_reimb_cols = ['InscClaimAmtReimbursed', 'DeductibleAmtPaid']
analyze_reimbursement(out_df, out_reimb_cols, label="Outpatient")

ben_reimb_cols = [
    'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
    'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt'
]
analyze_reimbursement(ben_df, ben_reimb_cols, label="Beneficiary Annual Amounts")


Inpatient Reimbursement:
------------------------------------------------------------
InscClaimAmtReimbursed:
  Missing: 0 (0.00%)
  Min: 0.00
  Max: 125,000.00
  Mean: 10,087.88
  Median: 7,000.00
DeductibleAmtPaid:
  Missing: 899 (2.22%)

Outpatient Reimbursement:
------------------------------------------------------------
InscClaimAmtReimbursed:
  Missing: 0 (0.00%)
  Min: 0.00
  Max: 102,500.00
  Mean: 286.33
  Median: 80.00
DeductibleAmtPaid:
  Missing: 0 (0.00%)
  Min: 0.00
  Max: 897.00
  Mean: 2.78
  Median: 0.00

Beneficiary Annual Amounts Reimbursement:
------------------------------------------------------------
IPAnnualReimbursementAmt:
  Missing: 0 (0.00%)
  Min: -8,000.00
  Max: 161,470.00
  Mean: 3,660.35
  Median: 0.00
IPAnnualDeductibleAmt:
  Missing: 0 (0.00%)
  Min: 0.00
  Max: 38,272.00
  Mean: 399.85
  Median: 0.00
OPAnnualReimbursementAmt:
  Missing: 0 (0.00%)
  Min: -70.00
  Max: 102,960.00
  Mean: 1,298.22
  Median: 570.00
OPAnnualDeductibleAmt:
  Missing: 0 (

In [22]:
chronic_cols = [col for col in ben_df.columns if 'ChronicCond' in col]
print(f"\nChronic Condition Flags ({len(chronic_cols)} columns):")
print("-" * 60)

chronic_missing = []
for col in sorted(chronic_cols):
    missing_count = ben_df[col].isna().sum()
    missing_pct = (missing_count / len(ben_df) * 100)
    unique_values = ben_df[col].value_counts().to_dict()
    chronic_missing.append({
        'Column': col,
        'Missing_Count': missing_count,
        'Missing_Percentage': f"{missing_pct:.2f}%",
        'Value_Counts': unique_values
    })
    print(f"{col}:")
    print(f"  Missing: {missing_count} ({missing_pct:.2f}%)")
    print(f"  Value distribution: {unique_values}")

# Check if all are binary (1 and 2) (include any missing values)
print(f"\nChronic Condition Value Ranges:")
print("-" * 60)
for col in sorted(chronic_cols):
    unique_vals = sorted(ben_df[col].unique())
    print(f"{col}: {unique_vals}")


Chronic Condition Flags (11 columns):
------------------------------------------------------------
ChronicCond_Alzheimer:
  Missing: 0 (0.00%)
  Value distribution: {2: 92530, 1: 46026}
ChronicCond_Cancer:
  Missing: 0 (0.00%)
  Value distribution: {2: 121935, 1: 16621}
ChronicCond_Depression:
  Missing: 0 (0.00%)
  Value distribution: {2: 89296, 1: 49260}
ChronicCond_Diabetes:
  Missing: 0 (0.00%)
  Value distribution: {1: 83391, 2: 55165}
ChronicCond_Heartfailure:
  Missing: 0 (0.00%)
  Value distribution: {2: 70154, 1: 68402}
ChronicCond_IschemicHeart:
  Missing: 0 (0.00%)
  Value distribution: {1: 93644, 2: 44912}
ChronicCond_KidneyDisease:
  Missing: 0 (0.00%)
  Value distribution: {2: 95277, 1: 43279}
ChronicCond_ObstrPulmonary:
  Missing: 0 (0.00%)
  Value distribution: {2: 105697, 1: 32859}
ChronicCond_Osteoporasis:
  Missing: 0 (0.00%)
  Value distribution: {2: 100497, 1: 38059}
ChronicCond_rheumatoidarthritis:
  Missing: 0 (0.00%)
  Value distribution: {2: 102972, 1: 35584}


In [23]:
print("\nDate of Birth (DOB):")
print("-" * 60)
dob_missing = ben_df['DOB'].isna().sum()
print(f"Missing: {dob_missing} ({dob_missing/len(ben_df)*100:.2f}%)")
print(f"Non-null: {ben_df['DOB'].notna().sum()}")
print(f"Sample values: {ben_df['DOB'].head(5).tolist()}")

print("\nDate of Death (DOD):")
print("-" * 60)
dod_missing = ben_df['DOD'].isna().sum()
print(f"Missing: {dod_missing} ({dod_missing/len(ben_df)*100:.2f}%)")
print(f"Non-null: {ben_df['DOD'].notna().sum()}")
print(f"Sample non-null values: {ben_df['DOD'].dropna().head(5).tolist()}")

# Calculate percentage of deceased beneficiaries
deceased_count = ben_df['DOD'].notna().sum()
print(f"\nDeceased beneficiaries: {deceased_count} ({deceased_count/len(ben_df)*100:.2f}%)")
print(f"Living beneficiaries: {ben_df['DOD'].isna().sum()} ({ben_df['DOD'].isna().sum()/len(ben_df)*100:.2f}%)")


Date of Birth (DOB):
------------------------------------------------------------
Missing: 0 (0.00%)
Non-null: 138556
Sample values: ['1943-01-01', '1936-09-01', '1936-08-01', '1922-07-01', '1935-09-01']

Date of Death (DOD):
------------------------------------------------------------
Missing: 137135 (98.97%)
Non-null: 1421
Sample non-null values: ['2009-12-01', '2009-09-01', '2009-09-01', '2009-05-01', '2009-12-01']

Deceased beneficiaries: 1421 (1.03%)
Living beneficiaries: 137135 (98.97%)


In [24]:
def analyze_cardinality(df, df_name, threshold=100):
    print(f"\n{df_name} Dataset - Columns with High Cardinality (>{threshold} unique values):")
    print("-" * 60)
    
    cardinality = []
    for col in df.columns:
        n_unique = df[col].nunique()
        cardinality.append({
            'Column': col,
            'Unique_Values': n_unique,
            'Total_Rows': len(df),
            'Cardinality_Ratio': f"{(n_unique/len(df)*100):.2f}%"
        })
    
    cardinality_df = pd.DataFrame(cardinality)
    cardinality_df = cardinality_df.sort_values('Unique_Values', ascending=False)
    
    # Show all columns above threshold
    high_card = cardinality_df[cardinality_df['Unique_Values'] > threshold]
    if len(high_card) > 0:
        print(high_card.to_string(index=False))
    else:
        print(f"  No columns with >{threshold} unique values")
    
    return cardinality_df

# Analyze each dataset
ben_card = analyze_cardinality(ben_df, "Beneficiary")
inp_card = analyze_cardinality(inp_df, "Inpatient")
out_card = analyze_cardinality(out_df, "Outpatient")
label_card = analyze_cardinality(label_df, "Labels")


Beneficiary Dataset - Columns with High Cardinality (>100 unique values):
------------------------------------------------------------
                  Column  Unique_Values  Total_Rows Cardinality_Ratio
                  BeneID         138556      138556           100.00%
IPAnnualReimbursementAmt           3004      138556             2.17%
OPAnnualReimbursementAmt           2078      138556             1.50%
                     DOB            900      138556             0.65%
   OPAnnualDeductibleAmt            789      138556             0.57%
                  County            314      138556             0.23%
   IPAnnualDeductibleAmt            147      138556             0.11%

Inpatient Dataset - Columns with High Cardinality (>100 unique values):
------------------------------------------------------------
                Column  Unique_Values  Total_Rows Cardinality_Ratio
               ClaimID          40474       40474           100.00%
                BeneID          31

In [25]:
physician_cols = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']

print("\nInpatient Physicians:")
print("-" * 60)
for col in physician_cols:
    if col in inp_df.columns:
        n_unique = inp_df[col].nunique()
        n_missing = inp_df[col].isna().sum()
        print(f"{col}:")
        print(f"  Unique values: {n_unique:,}")
        print(f"  Missing: {n_missing:,} ({n_missing/len(inp_df)*100:.2f}%)")
        print(f"  Non-null: {inp_df[col].notna().sum():,}")

print("\nOutpatient Physicians:")
print("-" * 60)
for col in physician_cols:
    if col in out_df.columns:
        n_unique = out_df[col].nunique()
        n_missing = out_df[col].isna().sum()
        print(f"{col}:")
        print(f"  Unique values: {n_unique:,}")
        print(f"  Missing: {n_missing:,} ({n_missing/len(out_df)*100:.2f}%)")
        print(f"  Non-null: {out_df[col].notna().sum():,}")


Inpatient Physicians:
------------------------------------------------------------
AttendingPhysician:
  Unique values: 11,604
  Missing: 112 (0.28%)
  Non-null: 40,362
OperatingPhysician:
  Unique values: 8,287
  Missing: 16,644 (41.12%)
  Non-null: 23,830
OtherPhysician:
  Unique values: 2,877
  Missing: 35,784 (88.41%)
  Non-null: 4,690

Outpatient Physicians:
------------------------------------------------------------
AttendingPhysician:
  Unique values: 74,109
  Missing: 1,396 (0.27%)
  Non-null: 516,341
OperatingPhysician:
  Unique values: 28,532
  Missing: 427,120 (82.50%)
  Non-null: 90,617
OtherPhysician:
  Unique values: 44,388
  Missing: 322,691 (62.33%)
  Non-null: 195,046


In [32]:
def analyze_diag_proc_cardinality(df, cols, dataset):
    print(f"\n{dataset} Cardinality:")
    print("-" * 60)

    card_stats = []

    for col in sorted(cols):
        n_unique = df[col].dropna().nunique()
        n_non_null = df[col].notna().sum()

        card_stats.append({
            "Column": col,
            "Unique_Values": n_unique,
            "Non_Null_Count": n_non_null
        })

    card_df = pd.DataFrame(card_stats)
    print(card_df.to_string(index=False))

    return card_df

inp_diag_cols = [col for col in inp_df.columns if 'ClmDiagnosisCode' in col]
out_diag_cols = [col for col in out_df.columns if 'ClmDiagnosisCode' in col]
inp_proc_cols = [col for col in inp_df.columns if 'ClmProcedureCode' in col]
out_proc_cols = [col for col in out_df.columns if 'ClmProcedureCode' in col]

inp_diag_card_df = analyze_diag_proc_cardinality(inp_df, inp_diag_cols, "Inpatient Diagnosis Codes")
out_diag_card_df = analyze_diag_proc_cardinality(out_df, out_diag_cols, "Outpatient Diagnosis Codes")
inp_proc_card_df = analyze_diag_proc_cardinality(inp_df, inp_proc_cols, "Inpatient Procedure Codes")
out_proc_card_df = analyze_diag_proc_cardinality(out_df, out_proc_cols, "Outpatient Procedure Codes")


Inpatient Diagnosis Codes Cardinality:
------------------------------------------------------------
             Column  Unique_Values  Non_Null_Count
 ClmDiagnosisCode_1           2254           40474
ClmDiagnosisCode_10            952            3927
 ClmDiagnosisCode_2           2439           40248
 ClmDiagnosisCode_3           2427           39798
 ClmDiagnosisCode_4           2441           38940
 ClmDiagnosisCode_5           2374           37580
 ClmDiagnosisCode_6           2358           35636
 ClmDiagnosisCode_7           2310           33216
 ClmDiagnosisCode_8           2243           30532
 ClmDiagnosisCode_9           2094           26977

Outpatient Diagnosis Codes Cardinality:
------------------------------------------------------------
             Column  Unique_Values  Non_Null_Count
 ClmDiagnosisCode_1          10354          507284
ClmDiagnosisCode_10            495            1083
 ClmDiagnosisCode_2           5056          322357
 ClmDiagnosisCode_3           44

In [33]:
print("\nClaimID:")
print("-" * 60)
inp_claimid_unique = inp_df['ClaimID'].nunique()
out_claimid_unique = out_df['ClaimID'].nunique()
all_claims_combined = pd.concat([inp_df[['ClaimID']], out_df[['ClaimID']]], ignore_index=True)
all_claimid_unique = all_claims_combined['ClaimID'].nunique()

print(f"Inpatient unique ClaimIDs: {inp_claimid_unique:,}")
print(f"Outpatient unique ClaimIDs: {out_claimid_unique:,}")
print(f"Total unique ClaimIDs (combined): {all_claimid_unique:,}")
print(f"Total claim rows: {len(all_claims_combined):,}")
print(f"ClaimID overlap between Inpatient and Outpatient: {inp_claimid_unique + out_claimid_unique - all_claimid_unique:,}")

print("\nBeneID:")
print("-" * 60)
ben_beneid_unique = ben_df['BeneID'].nunique()
inp_beneid_unique = inp_df['BeneID'].nunique()
out_beneid_unique = out_df['BeneID'].nunique()
all_claims_beneid_unique = all_claims_combined['BeneID'].nunique() if 'BeneID' in all_claims_combined.columns else 0

print(f"Beneficiary unique BeneIDs: {ben_beneid_unique:,}")
print(f"Inpatient unique BeneIDs: {inp_beneid_unique:,}")
print(f"Outpatient unique BeneIDs: {out_beneid_unique:,}")
print(f"Total unique BeneIDs in all claims: {all_claims_beneid_unique:,}")

# Provider cardinality
print("\nProvider:")
print("-" * 60)
inp_provider_unique = inp_df['Provider'].nunique()
out_provider_unique = out_df['Provider'].nunique()
all_providers = set(inp_df['Provider'].unique()) | set(out_df['Provider'].unique())
all_provider_unique = len(all_providers)
label_provider_unique = label_df['Provider'].nunique()

print(f"Inpatient unique Providers: {inp_provider_unique:,}")
print(f"Outpatient unique Providers: {out_provider_unique:,}")
print(f"Total unique Providers in claims: {all_provider_unique:,}")
print(f"Unique Providers in labels: {label_provider_unique:,}")


ClaimID:
------------------------------------------------------------
Inpatient unique ClaimIDs: 40,474
Outpatient unique ClaimIDs: 517,737
Total unique ClaimIDs (combined): 558,211
Total claim rows: 558,211
ClaimID overlap between Inpatient and Outpatient: 0

BeneID:
------------------------------------------------------------
Beneficiary unique BeneIDs: 138,556
Inpatient unique BeneIDs: 31,289
Outpatient unique BeneIDs: 133,980
Total unique BeneIDs in all claims: 0

Provider:
------------------------------------------------------------
Inpatient unique Providers: 2,092
Outpatient unique Providers: 5,012
Total unique Providers in claims: 5,410
Unique Providers in labels: 5,410


In [28]:
inp_proc_df

,Column,Missing_Count,Missing_Percentage,Non_Null_Count
0,ClmProcedureCode_1,17326,42.81%,23148
1,ClmProcedureCode_2,35020,86.52%,5454
2,ClmProcedureCode_3,39509,97.62%,965
3,ClmProcedureCode_4,40358,99.71%,116
4,ClmProcedureCode_5,40465,99.98%,9
5,ClmProcedureCode_6,40474,100.00%,0
